In [4]:
from dataclasses import dataclass
from typing import Union
from pydantic_graph import BaseNode, End, Graph, GraphRunContext

In [5]:
@dataclass
class CounterState:
    value: int

In [6]:
@dataclass
class Increment(BaseNode[CounterState]):
    async def run(self, ctx: GraphRunContext) -> "CheckIfDone":
        ctx.state.value += 1
        return CheckIfDone()


@dataclass
class CheckIfDone(BaseNode[CounterState, None, int]):
    async def run(self, ctx: GraphRunContext) -> Union[Increment, End[int]]:
        if ctx.state.value >= 3:
            return End(ctx.state.value)
        return Increment()

In [8]:
graph = Graph(nodes=(Increment, CheckIfDone))

result = await graph.run(Increment(), state=CounterState(value=0))
result

GraphRunResult(output=3, state=CounterState(value=3))